In [1]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt#visualization
# from PIL import  Image
# %matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
# import plotly.offline as py#visualization
# py.init_notebook_mode(connected=True)#visualization
# import plotly.graph_objs as go#visualization
# import plotly.tools as tls#visualization
# import plotly.figure_factory as ff#visualization

In [3]:
df = pd.read_csv(r"meg_telco_data_orig-Copy1.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Check columns types
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
print ("Rows     : " ,df.shape[0])
print ("Columns  : " ,df.shape[1])
print ("\nFeatures : \n" ,df.columns.tolist())
print ("\nMissing values :  ", df.isnull().sum().values.sum())
print ("\nUnique values :  \n",df.nunique())

Rows     :  7043
Columns  :  21

Features : 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Missing values :   0

Unique values :  
 customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64


In [5]:
(df['TotalCharges']==" ").sum()

11

In [6]:
df['TotalCharges'] = df["TotalCharges"].replace(" ",np.nan)
print(round((df['TotalCharges'].isnull().sum()/df.shape[0])*100,2),'% of missing values')

0.16 % of missing values


In [7]:
df = df[df["TotalCharges"].notnull()] #drop missing values
df = df.reset_index()[df.columns]

In [8]:
df.shape

(7032, 21)

In [9]:
print(round((df['TotalCharges'].isnull().sum()/df.shape[0])*100,2),'% of missing values')

0.0 % of missing values


In [10]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [11]:
df.OnlineSecurity.value_counts()

No                     3497
Yes                    2015
No internet service    1520
Name: OnlineSecurity, dtype: int64

In [12]:
# # make all column names lowercase
# df.columns = map(str.lower, df.columns)
# df.columns

In [13]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_notebook_iframe()

# **Our Data**
We have 7043 rows (each representing a unique customer) with 21 columns:     19 features, 1 target feature (Churn)
### **Target:**
- Churn (binary)
    
### **Nominal Features:**
- Tenure
- MonthlyCharges
- TotalCharges
    
### **Categorical Features:**
- Gender
- SeniorCitizen
- Dependents
- PhoneService
- MultipleLines
- InternetService
- OnlineSecurity
- OnlineBackup
- DeviceProtection
- TechSupport
- StreamingTV
- StreamingMovies
- Contract
- PaperlessBilling
- PaymentMethod
    

In [47]:
#convert to float type
df["TotalCharges"] = df["TotalCharges"].astype(float)

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})
    
#replace values
df["SeniorCitizen"] = df["SeniorCitizen"].replace({1:"Yes",0:"No"})

#Tenure to categorical column
def tenure_lab(df) :
    
    if df["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (df["tenure"] > 12) & (df["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (df["tenure"] > 24) & (df["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (df["tenure"] > 48) & (df["tenure"] <= 60) :
        return "Tenure_48-60"
    elif df["tenure"] > 60 :
        return "Tenure_gt_60"
df["tenure_group"] = df.apply(lambda df:tenure_lab(df),
                                      axis = 1)

#Separating churn and non churn customers
churn     = df[df["Churn"] == "Yes"]
not_churn = df[df["Churn"] == "No"]

In [48]:
df.drop('customerID',axis=1,inplace=True)

In [49]:
df1=df.copy()

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#Target columns
target_col = ["Churn"]

#categorical columns
cat_cols   = df.nunique()[df.nunique() < 6].keys().tolist()

#OR: 
# for i in cat_cols:
#     if i not in target_col:
#         cat.append(i)
        
cat_cols   = [x for x in cat_cols if x not in target_col]

#numerical columns
num_cols   = [x for x in df.columns if x not in cat_cols + target_col]

#Binary columns with 2 values
bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()

#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    df[i] = le.fit_transform(df[i])
    
#Duplicating columns for multi value columns
df = pd.get_dummies(data = df,columns = multi_cols )


In [51]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'tenure_group_Tenure_0-12', 'tenure_group_Tenure_12-24',
       'tenure_group_Tenure_24-48', 'tenure_group_Tenure_48-60',
       'tenure_group_Tenure_gt_60'],
      dtype='object')

In [52]:
#keeping n-1 dummy variables only
df.drop('MultipleLines_No phone service',axis=1,inplace=True)
df.drop('InternetService_No',axis=1,inplace=True)
df.drop('Contract_Two year',axis=1,inplace=True)
df.drop('PaymentMethod_Mailed check',axis=1,inplace=True)

In [53]:
# #Scaling Numerical columns
# std = StandardScaler()
# scaled = std.fit_transform(df[num_cols])
# scaled = pd.DataFrame(scaled,columns=num_cols)

# #MinMax scaler
# from sklearn.preprocessing import MinMaxScaler
# std = MinMaxScaler()
# scaled = std.fit_transform(df[num_cols])
# scaled = pd.DataFrame(scaled,columns=num_cols)

#what dave said :
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler() # import the scaler
# X_train = scaler.fit_transform(X_train) # learn and apply to train (from train)
# X_test = scaler.transform(X_test) # simply TRANSFORM/APPLY to the test data (no leakage!)


#dropping original values merging scaled values for numerical columns
# df_df_og = df.copy()
# df = df.drop(columns = num_cols,axis = 1)
# df = df.merge(scaled,left_index=True,right_index=True,how = "left")


In [54]:
df.dtypes

gender                                       int32
SeniorCitizen                                int32
Partner                                      int32
Dependents                                   int32
tenure                                       int64
PhoneService                                 int32
OnlineSecurity                               int32
OnlineBackup                                 int32
DeviceProtection                             int32
TechSupport                                  int32
StreamingTV                                  int32
StreamingMovies                              int32
PaperlessBilling                             int32
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int32
MultipleLines_No                             uint8
MultipleLines_Yes                            uint8
InternetService_DSL                          uint8
InternetService_Fiber optic    

In [55]:
df.to_csv('telco_clean.csv',index=False)

In [56]:
from sklearn.metrics import confusion_matrix

Testing :

In [57]:
df=df[['gender', 'SeniorCitizen', 'Partner','tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'PaperlessBilling',
       'Churn', 'InternetService_DSL',
       'InternetService_Fiber optic']]

In [58]:
df

,gender,SeniorCitizen,Partner,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,Churn,InternetService_DSL,InternetService_Fiber optic
0,0,0,1,1,0,0,1,0,0,1,0,1,0
1,1,0,0,34,1,1,0,1,0,0,0,1,0
2,1,0,0,2,1,1,1,0,0,1,1,1,0
3,1,0,0,45,0,1,0,1,1,0,0,1,0
4,0,0,0,2,1,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,1,0,1,24,1,1,0,1,1,1,0,1,0
7028,0,0,1,72,1,0,1,1,0,1,0,0,1
7029,0,0,1,11,0,1,0,0,0,1,0,1,0
7030,1,1,1,4,1,0,0,0,0,1,1,0,1


In [59]:
cat_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'tenure_group']

In [60]:
num_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [61]:
multi_cols

['MultipleLines',
 'InternetService',
 'Contract',
 'PaymentMethod',
 'tenure_group']

In [62]:
bin_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling',
 'Churn']

In [63]:
df.PhoneService

0       0
1       1
2       1
3       0
4       1
       ..
7027    1
7028    1
7029    0
7030    1
7031    1
Name: PhoneService, Length: 7032, dtype: int32

In [64]:
df1.PhoneService

0        No
1       Yes
2       Yes
3        No
4       Yes
       ... 
7027    Yes
7028    Yes
7029     No
7030    Yes
7031    Yes
Name: PhoneService, Length: 7032, dtype: object

In [65]:
df1.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'tenure_group'],
      dtype='object')

In [66]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'tenure', 'PhoneService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'PaperlessBilling', 'Churn', 'InternetService_DSL',
       'InternetService_Fiber optic'],
      dtype='object')